# Configuracion inicial

In [1]:
import os
os.chdir('D:/Compartida/Python 3')
os.getcwd()

import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline

import wlkfwdv6 as wk # Paquete creado en Valores-Bancolombia
import Importar as imp
import numpy as np
import pandas as pd
import pyfolio as pf

import warnings
warnings.filterwarnings("ignore")

import alphalensv3 as al

In [30]:
#IMPORTAR DATOS

simbolos = ["AAPL",
            "AMGN",
            "AMZN",
            "AXP",
            "BA",
            "BAC",
            "BMY",
            "BTI",
            "C",
            "CAT",
            "CL",
            "CSCO",
            "CVS",
            "CVX",
            "DIS",
            "EBAY",
            "FB",
            "FDX",
            "GE",
            "GOOGL",
            "GS",
            "HD",
            "HON",
            "HPE",
            "IBM",
            "INTC",
            "JNJ",
            "JPM",
            "KO",
            "LLY",
            "LUV",
            "MCD",
            "MMM",
            "MO",
#             "MON", # Aqui si se incluye Monsanto
            "MRK",
            "MS",
            "MSFT",
            "NKE",
            "ORCL",
            "OXY",
            "PFE",
            "PG",
            "PM",
            "QCOM",
            "SCCO",
            "SLB",
            "T",
            "UNH",
            "UNP",
            "UPS",
            "USB",
            "V",
            "WBA",
            "WFC",
            "WMT",
            "XOM"
            ]


años = [2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]

basedatos = imp.raw_pipeline(simbolos,años)
fin = '2018-10-10'
basedatos = basedatos[basedatos.index.get_level_values(0)<=fin]

# ALPHA # 1

In [3]:
import pdb

def alpha1(df,parameters):
    """
    df: Un Dataframe
    parameters: Una tupla con los valores de los parametros
    """

    def to_rt(serie):
        return serie/serie.shift(1)-1
    
    def stddev(serie):
        return serie.rolling(20).std()
    
    def ts_argmax(serie):
        return serie.rolling(5).apply(np.argmax)
    
    def rank(serie):
        return serie.rank()
    
    jon =df.copy()
    
    grouper = jon.index.get_level_values('symbol')
    grouper2 = jon.index.get_level_values('date')

    # Siempre debe estar la siguiente linea
    jon['ret']=jon.groupby(grouper)['Close'].apply(to_rt) # returns
    jon['ret']=jon['ret'].replace(np.nan,0) 
    
    jon['stddev'] = jon.groupby(grouper)['ret'].apply(stddev) # stddev(returns, 20)
    
    jon['returns < 0'] = np.where(jon['ret']<0,jon['stddev'],jon['Close']) # (returns < 0) ? stddev(returns, 20) : close)
    
    jon['(returns < 0)^2'] = jon['returns < 0']**2 # SignedPower(((returns < 0) ? stddev(returns, 20) : close), 2.)
    
    jon['Ts_ArgMax'] = jon.groupby(grouper)['(returns < 0)^2'].apply(ts_argmax) # Ts_ArgMax(SignedPower(((returns < 0) ? stddev(returns, 20) : close), 2.), 5)
    
    jon['factor'] = jon.groupby(grouper2)['Ts_ArgMax'].apply(rank)

    return jon

In [4]:
param = [1]
try1 = alpha1(basedatos,param)
try1

Open      High       Low     Close      Volume  \
date       symbol                                                       
2014-01-01 AAPL     79.1671   80.1829   79.1429   80.1457  55819372.0   
           AMGN    115.9100  115.9500  113.6700  114.0800   2362823.0   
           AMZN    394.5800  398.8300  393.8000  398.7900   1997051.0   
           AXP      89.8700   90.7900   89.5200   90.7300   3084500.0   
           BA      136.1600  137.0500  135.7400  136.4900   2096204.0   
           BAC      15.6000   15.6150   15.5100   15.5700  57230964.0   
           BMY      53.0500   53.3850   52.5900   53.1500   3994897.0   
           BTI      53.3700   53.7100   53.2665   53.7100    296444.0   
           C        51.9600   52.1200   51.8100   52.1100  10714210.0   
           CAT      91.0000   91.1799   90.4600   90.8100   3099524.0   
           CL       65.4200   65.5200   65.0010   65.2100   1827441.0   
           CSCO     22.2500   22.4700   22.1500   22.4300  33246316.0   
           CVS      71.8000   71.9750   71.3000   71.5700   2301615.0   
           CVX     124.6800  125.1700  123.9300  124.9100   3476249.0   
           DIS      76.2800   76.5000   75.8100   76.4000   4692382.0   
           EBAY     22.7485   23.1357   22.7232   23.0915   6365881.0   
           FB       54.1200   54.8600   53.9100   54.6490  43152127.0   
           FDX     143.0100  144.1300  143.0000  143.7700   1103045.0   
           GE       27.9000   28.0900   27.8300   28.0300  30312595.0   
           GOOGL   556.6590  561.0430  553.6660  560.8980   1358300.0   
           GS      176.2200  177.4400  176.2200  177.2600   1664338.0   
           HD       81.9900   82.4700   81.7150   82.3400   4493865.0   
           HON      86.5085   87.1653   86.3752   86.9845   1456498.0   
           HPE          NaN       NaN       NaN       NaN         NaN   
           IBM     186.4900  187.7900  186.3000  187.5700   3620413.0   
           INTC     25.8100   26.0400   25.7700   25.9550  22809682.0   
           JNJ      92.1700   92.2160   91.1510   91.5900   6348274.0   
           JPM      58.2000   58.5500   58.0500   58.4800  11021128.0   
           KO       41.1300   41.3870   40.9800   41.3100  11730170.0   
           LLY      51.1900   51.2600   50.8500   51.0000   2928655.0   
...                     ...       ...       ...       ...         ...   
2018-10-10 JNJ     139.3500  140.4600  137.7300  137.7300   7265642.0   
           JPM     114.7000  114.9500  111.4700  111.4700  23086389.0   
           KO       46.3000   46.7000   45.6700   45.6800  13917018.0   
           LLY     114.9600  115.3700  113.2500  113.3400   5602901.0   
           LUV      59.3600   59.3800   56.8800   57.3300   7945400.0   
           MCD     171.9900  172.0000  168.2200  168.3700   4190540.0   
           MMM     209.1900  209.1900  201.5500  202.1500   3525211.0   
           MO       63.2000   64.5000   62.8400   62.9100  11685268.0   
           MRK      72.4800   72.8900   70.3900   70.4500  12651947.0   
           MS       46.3400   46.4100   44.5150   44.6200  14143842.0   
           MSFT    111.2400  111.5000  105.7900  106.1600  61376315.0   
           NKE      79.6600   80.0400   74.7700   74.9400  12977843.0   
           ORCL     48.8100   48.9800   47.3700   47.4000  20885381.0   
           OXY      82.3800   82.5800   77.8600   77.9200   5059169.0   
           PFE      45.4500   45.7300   44.4800   44.5100  32017319.0   
           PG       81.8100   82.6800   81.4400   81.4400   9295840.0   
           PM       84.7500   86.2200   84.0300   84.6200   6532302.0   
           QCOM     68.7700   69.5600   66.6200   66.7300  22296820.0   
           SCCO     41.9900   42.0200   40.7700   41.3500    960720.0   
           SLB      62.7800   63.0400   60.6700   60.8200  10394052.0   
           T        33.6300   33.9900   32.8400   32.8500  45520118.0   
           UNH     270.6000  271.4200  262.5200  263.0800   4267341.0   
        

### Comentarios:

Este factor tiene la particularidad que es el mismo entre varias especies, lo que hace mas dificil la particion entre cuantiles. Ademas puede darse el caso de que un cuantil tenga toda su ponderacion en un solo activo lo que seria mas riesgoso. Esto tambien puede aumentar la varianza.

# ALPHA # 2

In [5]:
import pdb

def alpha2(df,parameters):
    """
    df: Un Dataframe
    parameters: Una tupla con los valores de los parametros
    """

    def to_rt(serie):
        return serie/serie.shift(1)-1
    
    def to_log(serie):
        return serie.apply(np.log)
    
    def to_delta(serie,w):
#         pdb.set_trace()
        delta = serie-serie.shift(w)
        return delta
    
    def to_corr(d,col1,col2,dfm):
        try3 = pd.DataFrame()
        for i in list(dfm.index.get_level_values(1).unique()):
            try2 = dfm[dfm.index.get_level_values(1)==i]
            try2['corr'] = try2[col1].rolling(d).corr(try2[col2])
            try3 = pd.concat([try3,try2],axis = 0)
        try3 = try3.sort_index(level = 0)
        return try3
    
    def rank(serie):
        return serie.rank()
    
    jon =df.copy()
    
    grouper = jon.index.get_level_values('symbol')
    grouper2 = jon.index.get_level_values('date')

    # Siempre debe estar la siguiente linea
    jon['ret']=jon.groupby(grouper)['Close'].apply(to_rt) # returns
    jon['ret']=jon['ret'].replace(np.nan,0) 
    
    jon['log(volume)'] = jon.groupby(grouper)['Volume'].apply(to_log) # log(volume)
    
    jon['delta(log(volume), 2)'] = jon.groupby(grouper)['log(volume)'].apply(to_delta,parameters[0]) # delta(log(volume), 2)
    
    jon['rank_volume'] = jon.groupby(grouper2)['delta(log(volume), 2)'].apply(rank) # rank(delta(log(volume), 2))
    
    jon['((close - open) / open)'] = (jon['Close']-jon['Open'])/jon['Open'] # ((close - open) / open)
    
#     pdb.set_trace()
    jon['rank_intradia'] = jon.groupby(grouper2)['((close - open) / open)'].apply(rank) # rank(((close - open) / open))
    
    jon = to_corr(parameters[1],'rank_intradia','rank_volume',jon) # correlation(rank(delta(log(volume), 2)), rank(((close - open) / open)), 6))
    
    jon['factor'] = jon['corr']*-1
    
    return jon

In [6]:
param = [1,6]
try1 = alpha2(basedatos,param)
try1

Open      High       Low     Close      Volume  \
date       symbol                                                       
2014-01-01 AAPL     79.1671   80.1829   79.1429   80.1457  55819372.0   
           AMGN    115.9100  115.9500  113.6700  114.0800   2362823.0   
           AMZN    394.5800  398.8300  393.8000  398.7900   1997051.0   
           AXP      89.8700   90.7900   89.5200   90.7300   3084500.0   
           BA      136.1600  137.0500  135.7400  136.4900   2096204.0   
           BAC      15.6000   15.6150   15.5100   15.5700  57230964.0   
           BMY      53.0500   53.3850   52.5900   53.1500   3994897.0   
           BTI      53.3700   53.7100   53.2665   53.7100    296444.0   
           C        51.9600   52.1200   51.8100   52.1100  10714210.0   
           CAT      91.0000   91.1799   90.4600   90.8100   3099524.0   
           CL       65.4200   65.5200   65.0010   65.2100   1827441.0   
           CSCO     22.2500   22.4700   22.1500   22.4300  33246316.0   
           CVS      71.8000   71.9750   71.3000   71.5700   2301615.0   
           CVX     124.6800  125.1700  123.9300  124.9100   3476249.0   
           DIS      76.2800   76.5000   75.8100   76.4000   4692382.0   
           EBAY     22.7485   23.1357   22.7232   23.0915   6365881.0   
           FB       54.1200   54.8600   53.9100   54.6490  43152127.0   
           FDX     143.0100  144.1300  143.0000  143.7700   1103045.0   
           GE       27.9000   28.0900   27.8300   28.0300  30312595.0   
           GOOGL   556.6590  561.0430  553.6660  560.8980   1358300.0   
           GS      176.2200  177.4400  176.2200  177.2600   1664338.0   
           HD       81.9900   82.4700   81.7150   82.3400   4493865.0   
           HON      86.5085   87.1653   86.3752   86.9845   1456498.0   
           HPE          NaN       NaN       NaN       NaN         NaN   
           IBM     186.4900  187.7900  186.3000  187.5700   3620413.0   
           INTC     25.8100   26.0400   25.7700   25.9550  22809682.0   
           JNJ      92.1700   92.2160   91.1510   91.5900   6348274.0   
           JPM      58.2000   58.5500   58.0500   58.4800  11021128.0   
           KO       41.1300   41.3870   40.9800   41.3100  11730170.0   
           LLY      51.1900   51.2600   50.8500   51.0000   2928655.0   
...                     ...       ...       ...       ...         ...   
2018-10-10 JNJ     139.3500  140.4600  137.7300  137.7300   7265642.0   
           JPM     114.7000  114.9500  111.4700  111.4700  23086389.0   
           KO       46.3000   46.7000   45.6700   45.6800  13917018.0   
           LLY     114.9600  115.3700  113.2500  113.3400   5602901.0   
           LUV      59.3600   59.3800   56.8800   57.3300   7945400.0   
           MCD     171.9900  172.0000  168.2200  168.3700   4190540.0   
           MMM     209.1900  209.1900  201.5500  202.1500   3525211.0   
           MO       63.2000   64.5000   62.8400   62.9100  11685268.0   
           MRK      72.4800   72.8900   70.3900   70.4500  12651947.0   
           MS       46.3400   46.4100   44.5150   44.6200  14143842.0   
           MSFT    111.2400  111.5000  105.7900  106.1600  61376315.0   
           NKE      79.6600   80.0400   74.7700   74.9400  12977843.0   
           ORCL     48.8100   48.9800   47.3700   47.4000  20885381.0   
           OXY      82.3800   82.5800   77.8600   77.9200   5059169.0   
           PFE      45.4500   45.7300   44.4800   44.5100  32017319.0   
           PG       81.8100   82.6800   81.4400   81.4400   9295840.0   
           PM       84.7500   86.2200   84.0300   84.6200   6532302.0   
           QCOM     68.7700   69.5600   66.6200   66.7300  22296820.0   
           SCCO     41.9900   42.0200   40.7700   41.3500    960720.0   
           SLB      62.7800   63.0400   60.6700   60.8200  10394052.0   
           T        33.6300   33.9900   32.8400   32.8500  45520118.0   
           UNH     270.6000  271.4200  262.5200  263.0800   4267341.0   
        

# ALPHA # 3

In [7]:
import pdb

def alpha3(df,parameters):
    """
    df: Un Dataframe
    parameters: Una tupla con los valores de los parametros
    """

    def to_rt(serie):
        return serie/serie.shift(1)-1
    
    def to_z(serie,w):
#         pdb.set_trace()
        return (serie-serie.rolling(w).mean())/serie.rolling(w).std()
    
    def to_corr(d,col1,col2,dfm):
        try3 = pd.DataFrame()
        for i in list(dfm.index.get_level_values(1).unique()):
            try2 = dfm[dfm.index.get_level_values(1)==i]
            try2['corr'] = try2[col1].rolling(d).corr(try2[col2])
            try3 = pd.concat([try3,try2],axis = 0)
        try3 = try3.sort_index(level = 0)
        return try3
    
    def rank(serie):
        return serie.rank()
    
    jon =df.copy()
    
    grouper = jon.index.get_level_values('symbol')
    grouper2 = jon.index.get_level_values('date')

    # Siempre debe estar la siguiente linea
    jon['ret']=jon.groupby(grouper)['Close'].apply(to_rt) # returns
    jon['ret']=jon['ret'].replace(np.nan,0) 
    
    jon['zopen'] = jon.groupby(grouper)['Open'].apply(to_z,parameters[0]) # open
    
    jon['zvolume'] = jon.groupby(grouper)['Volume'].apply(to_z,parameters[0]) # volume
    
    jon['rank(open)'] = jon.groupby(grouper2)['zopen'].apply(rank) # rank(open)
    
    jon['rank(volume)'] = jon.groupby(grouper2)['zvolume'].apply(rank) # rank(open)
    
    jon = to_corr(parameters[1],'rank(open)','rank(volume)',jon) # correlation(rank(open), rank(volume), 10))
    
    jon['factor'] = jon['corr']*-1 # (-1 * correlation(rank(open), rank(volume), 10))
    
    return jon

In [8]:
param = [60,10]
try1 = alpha2(basedatos,param)
try1

Open      High       Low     Close      Volume  \
date       symbol                                                       
2014-01-01 AAPL     79.1671   80.1829   79.1429   80.1457  55819372.0   
           AMGN    115.9100  115.9500  113.6700  114.0800   2362823.0   
           AMZN    394.5800  398.8300  393.8000  398.7900   1997051.0   
           AXP      89.8700   90.7900   89.5200   90.7300   3084500.0   
           BA      136.1600  137.0500  135.7400  136.4900   2096204.0   
           BAC      15.6000   15.6150   15.5100   15.5700  57230964.0   
           BMY      53.0500   53.3850   52.5900   53.1500   3994897.0   
           BTI      53.3700   53.7100   53.2665   53.7100    296444.0   
           C        51.9600   52.1200   51.8100   52.1100  10714210.0   
           CAT      91.0000   91.1799   90.4600   90.8100   3099524.0   
           CL       65.4200   65.5200   65.0010   65.2100   1827441.0   
           CSCO     22.2500   22.4700   22.1500   22.4300  33246316.0   
           CVS      71.8000   71.9750   71.3000   71.5700   2301615.0   
           CVX     124.6800  125.1700  123.9300  124.9100   3476249.0   
           DIS      76.2800   76.5000   75.8100   76.4000   4692382.0   
           EBAY     22.7485   23.1357   22.7232   23.0915   6365881.0   
           FB       54.1200   54.8600   53.9100   54.6490  43152127.0   
           FDX     143.0100  144.1300  143.0000  143.7700   1103045.0   
           GE       27.9000   28.0900   27.8300   28.0300  30312595.0   
           GOOGL   556.6590  561.0430  553.6660  560.8980   1358300.0   
           GS      176.2200  177.4400  176.2200  177.2600   1664338.0   
           HD       81.9900   82.4700   81.7150   82.3400   4493865.0   
           HON      86.5085   87.1653   86.3752   86.9845   1456498.0   
           HPE          NaN       NaN       NaN       NaN         NaN   
           IBM     186.4900  187.7900  186.3000  187.5700   3620413.0   
           INTC     25.8100   26.0400   25.7700   25.9550  22809682.0   
           JNJ      92.1700   92.2160   91.1510   91.5900   6348274.0   
           JPM      58.2000   58.5500   58.0500   58.4800  11021128.0   
           KO       41.1300   41.3870   40.9800   41.3100  11730170.0   
           LLY      51.1900   51.2600   50.8500   51.0000   2928655.0   
...                     ...       ...       ...       ...         ...   
2018-10-10 JNJ     139.3500  140.4600  137.7300  137.7300   7265642.0   
           JPM     114.7000  114.9500  111.4700  111.4700  23086389.0   
           KO       46.3000   46.7000   45.6700   45.6800  13917018.0   
           LLY     114.9600  115.3700  113.2500  113.3400   5602901.0   
           LUV      59.3600   59.3800   56.8800   57.3300   7945400.0   
           MCD     171.9900  172.0000  168.2200  168.3700   4190540.0   
           MMM     209.1900  209.1900  201.5500  202.1500   3525211.0   
           MO       63.2000   64.5000   62.8400   62.9100  11685268.0   
           MRK      72.4800   72.8900   70.3900   70.4500  12651947.0   
           MS       46.3400   46.4100   44.5150   44.6200  14143842.0   
           MSFT    111.2400  111.5000  105.7900  106.1600  61376315.0   
           NKE      79.6600   80.0400   74.7700   74.9400  12977843.0   
           ORCL     48.8100   48.9800   47.3700   47.4000  20885381.0   
           OXY      82.3800   82.5800   77.8600   77.9200   5059169.0   
           PFE      45.4500   45.7300   44.4800   44.5100  32017319.0   
           PG       81.8100   82.6800   81.4400   81.4400   9295840.0   
           PM       84.7500   86.2200   84.0300   84.6200   6532302.0   
           QCOM     68.7700   69.5600   66.6200   66.7300  22296820.0   
           SCCO     41.9900   42.0200   40.7700   41.3500    960720.0   
           SLB      62.7800   63.0400   60.6700   60.8200  10394052.0   
           T        33.6300   33.9900   32.8400   32.8500  45520118.0   
           UNH     270.6000  271.4200  262.5200  263.0800   4267341.0   
        

# ALPHA # 6

In [15]:
import pdb

def alpha6(df,parameters):
    """
    df: Un Dataframe
    parameters: Una tupla con los valores de los parametros
    """

    def to_rt(serie):
        return serie/serie.shift(1)-1
    
    def to_corr(d,col1,col2,dfm):
        try3 = pd.DataFrame()
        for i in list(dfm.index.get_level_values(1).unique()):
            try2 = dfm[dfm.index.get_level_values(1)==i]
            try2['corr'] = try2[col1].rolling(d).corr(try2[col2])
            try3 = pd.concat([try3,try2],axis = 0)
        try3 = try3.sort_index(level = 0)
        return try3    
    
    jon =df.copy()
    
    grouper = jon.index.get_level_values('symbol')
    grouper2 = jon.index.get_level_values('date')

    # Siempre debe estar la siguiente linea
    jon['ret']=jon.groupby(grouper)['Close'].apply(to_rt) # 
    jon['ret']=jon['ret'].replace(np.nan,0)
    
    jon = to_corr(parameters[0],'Open','Volume',jon) # correlation(open,volume,10)
    
    jon['factor'] = jon['corr']*-1 # (-1*correlation(open,volume,10))
    
    return jon

In [16]:
param = [10]
try1 = alpha6(basedatos,param)
try1

Open      High       Low     Close      Volume  \
date       symbol                                                       
2014-01-01 AAPL     79.1671   80.1829   79.1429   80.1457  55819372.0   
           AMGN    115.9100  115.9500  113.6700  114.0800   2362823.0   
           AMZN    394.5800  398.8300  393.8000  398.7900   1997051.0   
           AXP      89.8700   90.7900   89.5200   90.7300   3084500.0   
           BA      136.1600  137.0500  135.7400  136.4900   2096204.0   
           BAC      15.6000   15.6150   15.5100   15.5700  57230964.0   
           BMY      53.0500   53.3850   52.5900   53.1500   3994897.0   
           BTI      53.3700   53.7100   53.2665   53.7100    296444.0   
           C        51.9600   52.1200   51.8100   52.1100  10714210.0   
           CAT      91.0000   91.1799   90.4600   90.8100   3099524.0   
           CL       65.4200   65.5200   65.0010   65.2100   1827441.0   
           CSCO     22.2500   22.4700   22.1500   22.4300  33246316.0   
           CVS      71.8000   71.9750   71.3000   71.5700   2301615.0   
           CVX     124.6800  125.1700  123.9300  124.9100   3476249.0   
           DIS      76.2800   76.5000   75.8100   76.4000   4692382.0   
           EBAY     22.7485   23.1357   22.7232   23.0915   6365881.0   
           FB       54.1200   54.8600   53.9100   54.6490  43152127.0   
           FDX     143.0100  144.1300  143.0000  143.7700   1103045.0   
           GE       27.9000   28.0900   27.8300   28.0300  30312595.0   
           GOOGL   556.6590  561.0430  553.6660  560.8980   1358300.0   
           GS      176.2200  177.4400  176.2200  177.2600   1664338.0   
           HD       81.9900   82.4700   81.7150   82.3400   4493865.0   
           HON      86.5085   87.1653   86.3752   86.9845   1456498.0   
           HPE          NaN       NaN       NaN       NaN         NaN   
           IBM     186.4900  187.7900  186.3000  187.5700   3620413.0   
           INTC     25.8100   26.0400   25.7700   25.9550  22809682.0   
           JNJ      92.1700   92.2160   91.1510   91.5900   6348274.0   
           JPM      58.2000   58.5500   58.0500   58.4800  11021128.0   
           KO       41.1300   41.3870   40.9800   41.3100  11730170.0   
           LLY      51.1900   51.2600   50.8500   51.0000   2928655.0   
...                     ...       ...       ...       ...         ...   
2018-10-10 JNJ     139.3500  140.4600  137.7300  137.7300   7265642.0   
           JPM     114.7000  114.9500  111.4700  111.4700  23086389.0   
           KO       46.3000   46.7000   45.6700   45.6800  13917018.0   
           LLY     114.9600  115.3700  113.2500  113.3400   5602901.0   
           LUV      59.3600   59.3800   56.8800   57.3300   7945400.0   
           MCD     171.9900  172.0000  168.2200  168.3700   4190540.0   
           MMM     209.1900  209.1900  201.5500  202.1500   3525211.0   
           MO       63.2000   64.5000   62.8400   62.9100  11685268.0   
           MRK      72.4800   72.8900   70.3900   70.4500  12651947.0   
           MS       46.3400   46.4100   44.5150   44.6200  14143842.0   
           MSFT    111.2400  111.5000  105.7900  106.1600  61376315.0   
           NKE      79.6600   80.0400   74.7700   74.9400  12977843.0   
           ORCL     48.8100   48.9800   47.3700   47.4000  20885381.0   
           OXY      82.3800   82.5800   77.8600   77.9200   5059169.0   
           PFE      45.4500   45.7300   44.4800   44.5100  32017319.0   
           PG       81.8100   82.6800   81.4400   81.4400   9295840.0   
           PM       84.7500   86.2200   84.0300   84.6200   6532302.0   
           QCOM     68.7700   69.5600   66.6200   66.7300  22296820.0   
           SCCO     41.9900   42.0200   40.7700   41.3500    960720.0   
           SLB      62.7800   63.0400   60.6700   60.8200  10394052.0   
           T        33.6300   33.9900   32.8400   32.8500  45520118.0   
           UNH     270.6000  271.4200  262.5200  263.0800   4267341.0   
        

# ALPHA # 44

In [17]:
import pdb

def alpha44(df,parameters):
    """
    df: Un Dataframe
    parameters: Una tupla con los valores de los parametros
    """

    def to_rt(serie):
        return serie/serie.shift(1)-1
    
    def to_corr(d,col1,col2,dfm):
        try3 = pd.DataFrame()
        for i in list(dfm.index.get_level_values(1).unique()):
            try2 = dfm[dfm.index.get_level_values(1)==i]
            try2['corr'] = try2[col1].rolling(d).corr(try2[col2])
            try3 = pd.concat([try3,try2],axis = 0)
        try3 = try3.sort_index(level = 0)
        return try3    
    
    def rank(serie):
        return serie.rank()
    
    jon =df.copy()
    
    grouper = jon.index.get_level_values('symbol')
    grouper2 = jon.index.get_level_values('date')

    # Siempre debe estar la siguiente linea
    jon['ret']=jon.groupby(grouper)['Close'].apply(to_rt) # returns
    jon['ret']=jon['ret'].replace(np.nan,0)
    
    jon['rank(volume)'] = jon.groupby(grouper2)['Volume'].apply(rank) # rank(volume)
    
    jon = to_corr(parameters[0],'High','rank(volume)',jon) # correlation(high,rank(volume),5)
    
    jon['factor'] = jon['corr']*-1# (-1*correlation(high,rank(volume),5))
    
    return jon

In [18]:
param = [5]
try1 = alpha44(basedatos,param)
try1

Open      High       Low     Close      Volume  \
date       symbol                                                       
2014-01-01 AAPL     79.1671   80.1829   79.1429   80.1457  55819372.0   
           AMGN    115.9100  115.9500  113.6700  114.0800   2362823.0   
           AMZN    394.5800  398.8300  393.8000  398.7900   1997051.0   
           AXP      89.8700   90.7900   89.5200   90.7300   3084500.0   
           BA      136.1600  137.0500  135.7400  136.4900   2096204.0   
           BAC      15.6000   15.6150   15.5100   15.5700  57230964.0   
           BMY      53.0500   53.3850   52.5900   53.1500   3994897.0   
           BTI      53.3700   53.7100   53.2665   53.7100    296444.0   
           C        51.9600   52.1200   51.8100   52.1100  10714210.0   
           CAT      91.0000   91.1799   90.4600   90.8100   3099524.0   
           CL       65.4200   65.5200   65.0010   65.2100   1827441.0   
           CSCO     22.2500   22.4700   22.1500   22.4300  33246316.0   
           CVS      71.8000   71.9750   71.3000   71.5700   2301615.0   
           CVX     124.6800  125.1700  123.9300  124.9100   3476249.0   
           DIS      76.2800   76.5000   75.8100   76.4000   4692382.0   
           EBAY     22.7485   23.1357   22.7232   23.0915   6365881.0   
           FB       54.1200   54.8600   53.9100   54.6490  43152127.0   
           FDX     143.0100  144.1300  143.0000  143.7700   1103045.0   
           GE       27.9000   28.0900   27.8300   28.0300  30312595.0   
           GOOGL   556.6590  561.0430  553.6660  560.8980   1358300.0   
           GS      176.2200  177.4400  176.2200  177.2600   1664338.0   
           HD       81.9900   82.4700   81.7150   82.3400   4493865.0   
           HON      86.5085   87.1653   86.3752   86.9845   1456498.0   
           HPE          NaN       NaN       NaN       NaN         NaN   
           IBM     186.4900  187.7900  186.3000  187.5700   3620413.0   
           INTC     25.8100   26.0400   25.7700   25.9550  22809682.0   
           JNJ      92.1700   92.2160   91.1510   91.5900   6348274.0   
           JPM      58.2000   58.5500   58.0500   58.4800  11021128.0   
           KO       41.1300   41.3870   40.9800   41.3100  11730170.0   
           LLY      51.1900   51.2600   50.8500   51.0000   2928655.0   
...                     ...       ...       ...       ...         ...   
2018-10-10 JNJ     139.3500  140.4600  137.7300  137.7300   7265642.0   
           JPM     114.7000  114.9500  111.4700  111.4700  23086389.0   
           KO       46.3000   46.7000   45.6700   45.6800  13917018.0   
           LLY     114.9600  115.3700  113.2500  113.3400   5602901.0   
           LUV      59.3600   59.3800   56.8800   57.3300   7945400.0   
           MCD     171.9900  172.0000  168.2200  168.3700   4190540.0   
           MMM     209.1900  209.1900  201.5500  202.1500   3525211.0   
           MO       63.2000   64.5000   62.8400   62.9100  11685268.0   
           MRK      72.4800   72.8900   70.3900   70.4500  12651947.0   
           MS       46.3400   46.4100   44.5150   44.6200  14143842.0   
           MSFT    111.2400  111.5000  105.7900  106.1600  61376315.0   
           NKE      79.6600   80.0400   74.7700   74.9400  12977843.0   
           ORCL     48.8100   48.9800   47.3700   47.4000  20885381.0   
           OXY      82.3800   82.5800   77.8600   77.9200   5059169.0   
           PFE      45.4500   45.7300   44.4800   44.5100  32017319.0   
           PG       81.8100   82.6800   81.4400   81.4400   9295840.0   
           PM       84.7500   86.2200   84.0300   84.6200   6532302.0   
           QCOM     68.7700   69.5600   66.6200   66.7300  22296820.0   
           SCCO     41.9900   42.0200   40.7700   41.3500    960720.0   
           SLB      62.7800   63.0400   60.6700   60.8200  10394052.0   
           T        33.6300   33.9900   32.8400   32.8500  45520118.0   
           UNH     270.6000  271.4200  262.5200  263.0800   4267341.0   
        

# ALPHA # 53

In [22]:
import pdb

def alpha53(df,parameters):
    """
    df: Un Dataframe
    parameters: Una tupla con los valores de los parametros
    """

    def to_rt(serie):
        return serie/serie.shift(1)-1

    def to_delta(serie,w):
        delta = serie-serie.shift(w)
        return delta
    
    jon =df.copy()
    
    grouper = jon.index.get_level_values('symbol')
    grouper2 = jon.index.get_level_values('date')

    # Siempre debe estar la siguiente linea
    jon['ret']=jon.groupby(grouper)['Close'].apply(to_rt) # returns
    jon['ret']=jon['ret'].replace(np.nan,0)
    
    jon['cambios'] = ((jon['Close']-jon['Low'])-(jon['High']-jon['Close']))/(jon['Close']-jon['Low']+0.0001) # ((close-low)-(high-close))/(close-low)
    
    jon['delta(cambios)'] = jon.groupby(grouper)['cambios'].apply(to_delta,parameters[0]) # delta(((close-low)-(high-close))/(close-low),9)
    
    jon['factor'] = jon['delta(cambios)']*-1 #(-1*delta(((close-low)-(high-close))/(close-low),9))
    
    return jon

In [23]:
param = [9]
try1 = alpha53(basedatos,param)
try1

Open      High       Low     Close      Volume  \
date       symbol                                                       
2014-01-01 AAPL     79.1671   80.1829   79.1429   80.1457  55819372.0   
           AMGN    115.9100  115.9500  113.6700  114.0800   2362823.0   
           AMZN    394.5800  398.8300  393.8000  398.7900   1997051.0   
           AXP      89.8700   90.7900   89.5200   90.7300   3084500.0   
           BA      136.1600  137.0500  135.7400  136.4900   2096204.0   
           BAC      15.6000   15.6150   15.5100   15.5700  57230964.0   
           BMY      53.0500   53.3850   52.5900   53.1500   3994897.0   
           BTI      53.3700   53.7100   53.2665   53.7100    296444.0   
           C        51.9600   52.1200   51.8100   52.1100  10714210.0   
           CAT      91.0000   91.1799   90.4600   90.8100   3099524.0   
           CL       65.4200   65.5200   65.0010   65.2100   1827441.0   
           CSCO     22.2500   22.4700   22.1500   22.4300  33246316.0   
           CVS      71.8000   71.9750   71.3000   71.5700   2301615.0   
           CVX     124.6800  125.1700  123.9300  124.9100   3476249.0   
           DIS      76.2800   76.5000   75.8100   76.4000   4692382.0   
           EBAY     22.7485   23.1357   22.7232   23.0915   6365881.0   
           FB       54.1200   54.8600   53.9100   54.6490  43152127.0   
           FDX     143.0100  144.1300  143.0000  143.7700   1103045.0   
           GE       27.9000   28.0900   27.8300   28.0300  30312595.0   
           GOOGL   556.6590  561.0430  553.6660  560.8980   1358300.0   
           GS      176.2200  177.4400  176.2200  177.2600   1664338.0   
           HD       81.9900   82.4700   81.7150   82.3400   4493865.0   
           HON      86.5085   87.1653   86.3752   86.9845   1456498.0   
           HPE          NaN       NaN       NaN       NaN         NaN   
           IBM     186.4900  187.7900  186.3000  187.5700   3620413.0   
           INTC     25.8100   26.0400   25.7700   25.9550  22809682.0   
           JNJ      92.1700   92.2160   91.1510   91.5900   6348274.0   
           JPM      58.2000   58.5500   58.0500   58.4800  11021128.0   
           KO       41.1300   41.3870   40.9800   41.3100  11730170.0   
           LLY      51.1900   51.2600   50.8500   51.0000   2928655.0   
...                     ...       ...       ...       ...         ...   
2018-10-10 JNJ     139.3500  140.4600  137.7300  137.7300   7265642.0   
           JPM     114.7000  114.9500  111.4700  111.4700  23086389.0   
           KO       46.3000   46.7000   45.6700   45.6800  13917018.0   
           LLY     114.9600  115.3700  113.2500  113.3400   5602901.0   
           LUV      59.3600   59.3800   56.8800   57.3300   7945400.0   
           MCD     171.9900  172.0000  168.2200  168.3700   4190540.0   
           MMM     209.1900  209.1900  201.5500  202.1500   3525211.0   
           MO       63.2000   64.5000   62.8400   62.9100  11685268.0   
           MRK      72.4800   72.8900   70.3900   70.4500  12651947.0   
           MS       46.3400   46.4100   44.5150   44.6200  14143842.0   
           MSFT    111.2400  111.5000  105.7900  106.1600  61376315.0   
           NKE      79.6600   80.0400   74.7700   74.9400  12977843.0   
           ORCL     48.8100   48.9800   47.3700   47.4000  20885381.0   
           OXY      82.3800   82.5800   77.8600   77.9200   5059169.0   
           PFE      45.4500   45.7300   44.4800   44.5100  32017319.0   
           PG       81.8100   82.6800   81.4400   81.4400   9295840.0   
           PM       84.7500   86.2200   84.0300   84.6200   6532302.0   
           QCOM     68.7700   69.5600   66.6200   66.7300  22296820.0   
           SCCO     41.9900   42.0200   40.7700   41.3500    960720.0   
           SLB      62.7800   63.0400   60.6700   60.8200  10394052.0   
           T        33.6300   33.9900   32.8400   32.8500  45520118.0   
           UNH     270.6000  271.4200  262.5200  263.0800   4267341.0   
        

# ALPHA # 54

In [37]:
import pdb

def alpha54(df,parameters):
    """
    df: Un Dataframe
    parameters: Una tupla con los valores de los parametros
    """

    def to_rt(serie):
        return serie/serie.shift(1)-1

    def to_delta(serie,w):
        delta = serie-serie.shift(w)
        return delta
    
    jon =df.copy()
    
    grouper = jon.index.get_level_values('symbol')
    grouper2 = jon.index.get_level_values('date')

    # Siempre debe estar la siguiente linea
    jon['ret']=jon.groupby(grouper)['Close'].apply(to_rt) # returns
    jon['ret']=jon['ret'].replace(np.nan,0)
    
    jon['factor'] = (-1*(jon['Low']-jon['Close']+0.0001)*(jon['Open']**5))/((jon['Low']-jon['High']+0.0001)*(jon['Open']**5)) # ((-1*(low-close)*(open○^5))/((low-high)*(open^5)))
    
    jon['factor'] = jon['factor']*(jon['ret']+1) # modificacion para los paquetes
    
    return jon

In [38]:
param = [1]
try1 = alpha54(basedatos,param)
try1.loc[try1.index.get_level_values(0)=='2009-04-03']

Open      High       Low     Close       Volume  \
date       symbol                                                        
2009-04-03 AAPL     16.3129   16.5900   16.2171   16.5700  159086263.0   
           AMGN     47.8700   47.9700   46.0800   46.5700    8484790.0   
           AMZN     76.4200   78.3200   75.5000   78.1700    5810612.0   
           AXP      14.8000   15.3300   14.5200   15.3300   19196445.0   
           BA       37.2300   38.1200   36.3900   37.6900    8591254.0   
           BAC       7.0900    7.6100    6.6500    7.6000  348608863.0   
           BMY      20.6100   22.0900   19.9300   20.1700   21717316.0   
           BTI      23.4350   23.4800   23.0500   23.4400     266516.0   
           C        27.5000   28.5000   26.4000   28.5000   27997320.0   
           CAT      31.6000   32.7000   30.9200   32.1500   13674074.0   
           CL       30.7150   30.7525   29.9000   30.2200    7900220.0   
           CSCO     18.3000   18.5400   17.9500   18.1600   59474243.0   
           CVS      29.3500   29.5500   28.1500   28.4500   11747478.0   
           CVX      69.9000   70.9200   69.4200   70.4800   11575037.0   
           DIS      19.9100   20.3900   19.7800   20.0000   18478331.0   
           EBAY      5.8081    6.1574    5.7366    6.0228   18551513.0   
           FB           NaN       NaN       NaN       NaN          NaN   
           FDX      48.3100   50.1500   48.0900   50.0200    4319365.0   
           GE       10.8100   10.9900   10.7100   10.9400  123701474.0   
           GOOGL   182.4270  186.0400  179.1730  185.0690    3789784.0   
           GS      112.8000  119.7600  112.5500  119.4000   23692027.0   
           HD       24.6900   25.1250   24.3400   25.0000   20294404.0   
           HON      29.0646   29.4930   28.3792   28.7505    6310606.0   
           HPE          NaN       NaN       NaN       NaN          NaN   
           IBM      99.7300  102.6600   99.7100  102.2200   10964564.0   
           INTC     15.6300   15.9600   15.3500   15.9500   51142797.0   
           JNJ      53.0000   53.3000   51.7500   52.1500   15214462.0   
           JPM      27.9800   29.2800   27.5000   29.2800   80949489.0   
           KO       22.6850   22.8800   22.2400   22.4850   17890010.0   
           LLY      33.5300   33.7500   32.5200   33.0100    7124409.0   
           LUV       7.0500    7.1600    6.7000    6.8600    9624752.0   
           MCD      56.2000   56.6400   55.5600   56.6400    7175617.0   
           MMM      52.2300   52.4900   51.2500   52.1100    4931056.0   
           MO       16.0100   16.2100   15.9700   16.1600   13763823.0   
           MRK      27.0000   27.3100   26.0899   26.4600   15483967.0   
           MS       22.6000   24.1600   22.0700   24.0600   52638488.0   
           MSFT     19.1300   19.1500   18.4343   18.7500   81865036.0   
           NKE      12.5650   12.9725   12.5500   12.9675   14375788.0   
           ORCL     18.8800   19.4500   18.8700   19.2900   40966931.0   
           OXY      57.4972   59.0608   56.9792   57.9289    9030407.0   
           PFE      13.5900   13.7700   13.4500   13.5500   52922095.0   
           PG       49.5400   49.7500   48.7200   49.6300   14800187.0   
           PM       36.0400   36.3900   35.8000   36.2000   10122238.0   
           QCOM     41.4825   41.4825   40.6300   41.1900   18373881.0   
           SCCO     18.4526   19.2243   18.4031   18.7494    8327583.0   
           SLB      43.7400   45.4800   43.5600   45.0700   13504322.0   
           T        26.7200   26.9700   26.5300   26.8300   25398697.0   
           UNH      21.0500   21.5000   20.1400   20.5400    9655024.0   
           UNP      22.7100   23.0850   22.5500   23.0450   18263486.0   
           UPS      51.9300   52.8200   51.7500   52.7500    6790842.0   
           USB      15.2500   16.0000   14.9600   15.9700   24159649.0   
           V        13.8175   15.0400   13.8175   15.0200   57716360.0   
           WBA      27.3600

# ALPHA # 101

In [11]:
import pdb

def alpha101(df,parameters):
    """
    df: Un Dataframe
    parameters: Una tupla con los valores de los parametros
    """

    def to_rt(serie):
        return serie/serie.shift(1)-1
    
    jon =df.copy()
    
    grouper = jon.index.get_level_values('symbol')
    grouper2 = jon.index.get_level_values('date')

    # Siempre debe estar la siguiente linea
    jon['ret']=jon.groupby(grouper)['Close'].apply(to_rt) # returns
    jon['ret']=jon['ret'].replace(np.nan,0)
    
    jon['factor'] = (jon['Close']-jon['Open'])/((jon['High']-jon['Low'])+0.001)
    
    return jon

In [12]:
param = [1]
try1 = alpha101(basedatos,param)
try1

Open      High       Low     Close      Volume  \
date       symbol                                                       
2014-01-01 AAPL     79.1671   80.1829   79.1429   80.1457  55819372.0   
           AMGN    115.9100  115.9500  113.6700  114.0800   2362823.0   
           AMZN    394.5800  398.8300  393.8000  398.7900   1997051.0   
           AXP      89.8700   90.7900   89.5200   90.7300   3084500.0   
           BA      136.1600  137.0500  135.7400  136.4900   2096204.0   
           BAC      15.6000   15.6150   15.5100   15.5700  57230964.0   
           BMY      53.0500   53.3850   52.5900   53.1500   3994897.0   
           BTI      53.3700   53.7100   53.2665   53.7100    296444.0   
           C        51.9600   52.1200   51.8100   52.1100  10714210.0   
           CAT      91.0000   91.1799   90.4600   90.8100   3099524.0   
           CL       65.4200   65.5200   65.0010   65.2100   1827441.0   
           CSCO     22.2500   22.4700   22.1500   22.4300  33246316.0   
           CVS      71.8000   71.9750   71.3000   71.5700   2301615.0   
           CVX     124.6800  125.1700  123.9300  124.9100   3476249.0   
           DIS      76.2800   76.5000   75.8100   76.4000   4692382.0   
           EBAY     22.7485   23.1357   22.7232   23.0915   6365881.0   
           FB       54.1200   54.8600   53.9100   54.6490  43152127.0   
           FDX     143.0100  144.1300  143.0000  143.7700   1103045.0   
           GE       27.9000   28.0900   27.8300   28.0300  30312595.0   
           GOOGL   556.6590  561.0430  553.6660  560.8980   1358300.0   
           GS      176.2200  177.4400  176.2200  177.2600   1664338.0   
           HD       81.9900   82.4700   81.7150   82.3400   4493865.0   
           HON      86.5085   87.1653   86.3752   86.9845   1456498.0   
           HPE          NaN       NaN       NaN       NaN         NaN   
           IBM     186.4900  187.7900  186.3000  187.5700   3620413.0   
           INTC     25.8100   26.0400   25.7700   25.9550  22809682.0   
           JNJ      92.1700   92.2160   91.1510   91.5900   6348274.0   
           JPM      58.2000   58.5500   58.0500   58.4800  11021128.0   
           KO       41.1300   41.3870   40.9800   41.3100  11730170.0   
           LLY      51.1900   51.2600   50.8500   51.0000   2928655.0   
...                     ...       ...       ...       ...         ...   
2018-10-10 JNJ     139.3500  140.4600  137.7300  137.7300   7265642.0   
           JPM     114.7000  114.9500  111.4700  111.4700  23086389.0   
           KO       46.3000   46.7000   45.6700   45.6800  13917018.0   
           LLY     114.9600  115.3700  113.2500  113.3400   5602901.0   
           LUV      59.3600   59.3800   56.8800   57.3300   7945400.0   
           MCD     171.9900  172.0000  168.2200  168.3700   4190540.0   
           MMM     209.1900  209.1900  201.5500  202.1500   3525211.0   
           MO       63.2000   64.5000   62.8400   62.9100  11685268.0   
           MRK      72.4800   72.8900   70.3900   70.4500  12651947.0   
           MS       46.3400   46.4100   44.5150   44.6200  14143842.0   
           MSFT    111.2400  111.5000  105.7900  106.1600  61376315.0   
           NKE      79.6600   80.0400   74.7700   74.9400  12977843.0   
           ORCL     48.8100   48.9800   47.3700   47.4000  20885381.0   
           OXY      82.3800   82.5800   77.8600   77.9200   5059169.0   
           PFE      45.4500   45.7300   44.4800   44.5100  32017319.0   
           PG       81.8100   82.6800   81.4400   81.4400   9295840.0   
           PM       84.7500   86.2200   84.0300   84.6200   6532302.0   
           QCOM     68.7700   69.5600   66.6200   66.7300  22296820.0   
           SCCO     41.9900   42.0200   40.7700   41.3500    960720.0   
           SLB      62.7800   63.0400   60.6700   60.8200  10394052.0   
           T        33.6300   33.9900   32.8400   32.8500  45520118.0   
           UNH     270.6000  271.4200  262.5200  263.0800   4267341.0   
        

# MAR

In [9]:
import pdb

def mar(df,parameters):
    """
    df: Un Dataframe
    parameters: Una tupla con los valores de los parametros
    """
    
    jon =df.copy()
    
    grouper = jon.index.get_level_values('symbol')
    def to_rt(serie):
        return serie/serie.shift(1)-1
    
    # Siempre debe estar la siguiente linea
    jon['ret']=jon.groupby(grouper)['Close'].apply(to_rt)
    jon['ret']=jon['ret'].replace(np.nan,0)
    
    def to_mean(serie,w):
        return serie.rolling(w).mean()
    
    jon['FMA']=jon.groupby(grouper)['Close'].apply(to_mean,w = parameters[0])
    jon['SMA']=jon.groupby(grouper)['Close'].apply(to_mean,w = parameters[1])
    
    jon['factor'] = jon['FMA']/jon['SMA'] # distancia entre ambas ma

    return jon

In [10]:
param = [50,200]
try1 = mar(basedatos,param)
try1

Open      High       Low     Close      Volume  \
date       symbol                                                       
2014-01-01 AAPL     79.1671   80.1829   79.1429   80.1457  55819372.0   
           AMGN    115.9100  115.9500  113.6700  114.0800   2362823.0   
           AMZN    394.5800  398.8300  393.8000  398.7900   1997051.0   
           AXP      89.8700   90.7900   89.5200   90.7300   3084500.0   
           BA      136.1600  137.0500  135.7400  136.4900   2096204.0   
           BAC      15.6000   15.6150   15.5100   15.5700  57230964.0   
           BMY      53.0500   53.3850   52.5900   53.1500   3994897.0   
           BTI      53.3700   53.7100   53.2665   53.7100    296444.0   
           C        51.9600   52.1200   51.8100   52.1100  10714210.0   
           CAT      91.0000   91.1799   90.4600   90.8100   3099524.0   
           CL       65.4200   65.5200   65.0010   65.2100   1827441.0   
           CSCO     22.2500   22.4700   22.1500   22.4300  33246316.0   
           CVS      71.8000   71.9750   71.3000   71.5700   2301615.0   
           CVX     124.6800  125.1700  123.9300  124.9100   3476249.0   
           DIS      76.2800   76.5000   75.8100   76.4000   4692382.0   
           EBAY     22.7485   23.1357   22.7232   23.0915   6365881.0   
           FB       54.1200   54.8600   53.9100   54.6490  43152127.0   
           FDX     143.0100  144.1300  143.0000  143.7700   1103045.0   
           GE       27.9000   28.0900   27.8300   28.0300  30312595.0   
           GOOGL   556.6590  561.0430  553.6660  560.8980   1358300.0   
           GS      176.2200  177.4400  176.2200  177.2600   1664338.0   
           HD       81.9900   82.4700   81.7150   82.3400   4493865.0   
           HON      86.5085   87.1653   86.3752   86.9845   1456498.0   
           HPE          NaN       NaN       NaN       NaN         NaN   
           IBM     186.4900  187.7900  186.3000  187.5700   3620413.0   
           INTC     25.8100   26.0400   25.7700   25.9550  22809682.0   
           JNJ      92.1700   92.2160   91.1510   91.5900   6348274.0   
           JPM      58.2000   58.5500   58.0500   58.4800  11021128.0   
           KO       41.1300   41.3870   40.9800   41.3100  11730170.0   
           LLY      51.1900   51.2600   50.8500   51.0000   2928655.0   
...                     ...       ...       ...       ...         ...   
2018-10-10 JNJ     139.3500  140.4600  137.7300  137.7300   7265642.0   
           JPM     114.7000  114.9500  111.4700  111.4700  23086389.0   
           KO       46.3000   46.7000   45.6700   45.6800  13917018.0   
           LLY     114.9600  115.3700  113.2500  113.3400   5602901.0   
           LUV      59.3600   59.3800   56.8800   57.3300   7945400.0   
           MCD     171.9900  172.0000  168.2200  168.3700   4190540.0   
           MMM     209.1900  209.1900  201.5500  202.1500   3525211.0   
           MO       63.2000   64.5000   62.8400   62.9100  11685268.0   
           MRK      72.4800   72.8900   70.3900   70.4500  12651947.0   
           MS       46.3400   46.4100   44.5150   44.6200  14143842.0   
           MSFT    111.2400  111.5000  105.7900  106.1600  61376315.0   
           NKE      79.6600   80.0400   74.7700   74.9400  12977843.0   
           ORCL     48.8100   48.9800   47.3700   47.4000  20885381.0   
           OXY      82.3800   82.5800   77.8600   77.9200   5059169.0   
           PFE      45.4500   45.7300   44.4800   44.5100  32017319.0   
           PG       81.8100   82.6800   81.4400   81.4400   9295840.0   
           PM       84.7500   86.2200   84.0300   84.6200   6532302.0   
           QCOM     68.7700   69.5600   66.6200   66.7300  22296820.0   
           SCCO     41.9900   42.0200   40.7700   41.3500    960720.0   
           SLB      62.7800   63.0400   60.6700   60.8200  10394052.0   
           T        33.6300   33.9900   32.8400   32.8500  45520118.0   
           UNH     270.6000  271.4200  262.5200  263.0800   4267341.0   
        